# **DeepSlice notebook**
this is an example notebook designed to teach you how to use DeepSlice 



In [1]:
import os

In [2]:
#first lets ensure we are in the DeepSlice parent directory
# if we are in examples/example_notebooks, we need to go up two directories
os.chdir('/home/onurserce/Data/Helmholz/241025_20241025_OS_SN43_555cFos_647NeuN_horizontal_09-48-56/fused_ch0_8bit_rotated_for_deepslice')

## **Import and Build the model**

In [3]:
from DeepSlice import DSModel

2024-11-21 19:29:29.929143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 19:29:29.929180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 19:29:29.930626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 19:29:29.937843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 19:29:30.709502: W tensorflow/compiler/tf2

In [4]:
species = 'mouse' #available species are 'mouse' and 'rat'

Model = DSModel(species)

2024-11-21 19:29:31.387033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-21 19:29:31.440255: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## **Make Predictions & Save Results**

Next we point our model towards a folder which contains images we would like to align. <br> 


* ensemble uses multiple models to make predictions, test this out on your data as it is not better in all cases (though takes twice as long to run)

* section numbers tells the model whether your images have section numbers in the file name, if True they should be included as _sXXX where X is the three digit section number

In [5]:
# sections are in reverse order, so the section numbers need to be changed

import shutil
from tqdm import tqdm

folderpath = '/home/onurserce/Data/Helmholz/241025_20241025_OS_SN43_555cFos_647NeuN_horizontal_09-48-56/fused_ch0_8bit_rotated_for_deepslice'
sections = [file for file in os.listdir(folderpath) if file.endswith('.png')]


last_section = 7577
reversed_folder = os.path.join(folderpath, 'reversed')
os.makedirs(reversed_folder, exist_ok=True)

for section in tqdm(sections):
    original_name = section
    section_number = int(original_name.split('_s')[1].split('.png')[0])
    new_number = last_section - section_number
    if new_number % 10 == 0:
        new_name = 'rev_sec_s' + str(new_number) + '.png'
        # copy the original section with the new numbering
        src = os.path.join(folderpath, original_name)
        dst = os.path.join(reversed_folder, new_name)
        shutil.copyfile(src, dst)

100%|██████████| 4801/4801 [00:00<00:00, 13901.54it/s]


In [6]:
#this cell should take about 1 second per image
folderpath = '/home/onurserce/Data/Helmholz/241025_20241025_OS_SN43_555cFos_647NeuN_horizontal_09-48-56/fused_ch0_8bit_rotated_for_deepslice/reversed'
Model.predict(folderpath, ensemble=True, section_numbers=True)

Found 480 validated image filenames.
480/480 [==============================] - 43s 89ms/step


**Angle propagation**
this tells the model to average the predicted angles (since your sections are cut from a block they should all share an angle)

In [7]:
Model.propagate_angles()

DV angle: 0.03410844959385597
ML angle: -1.1128794965658675
DV angle: 0.03504583326875531
ML angle: -1.1128794635737318


<h2>Use section number information</h2>
If you have included section numbers in your filenames we can do one of two things.<br>
You need to know how accurate your section numbers are, if they are simply the order the sections were cut you can use<br>

**Model.enforce_index_order()** <br>

To reorder your sections according to their number. If your section numbers are the precise index which the sections were cut (ie; 1, 2, 4, indicates that section 3 has been left out of the series) You can use <br>

**Model.enforce_index_spacing()**
<br>

If you know the exact Thickness of your sectionsin microns you can include this here as well, but don't worry, if you dont include it we will estimate and tell you our guess. <br>
**Model.enforce_index_spacing(section_thickness = 25)**

In [8]:
Model.enforce_index_order()

In [9]:
Model.enforce_index_spacing()

predicted thickness is -2.4592737282224912µm


The next cell will save both a CSV file and QuickNII XML file to the current directory. If you wish to view it in quickNII simply download the xml file and brain images and place the csv in the same directory as your images.

In [10]:
Model.save_predictions(folderpath + 'MyResults')

480
saving to /home/onurserce/Data/Helmholz/241025_20241025_OS_SN43_555cFos_647NeuN_horizontal_09-48-56/fused_ch0_8bit_rotated_for_deepslice/reversedMyResults.json
